In [76]:
import requests
import lxml.html
from xml.etree.ElementTree import *
import sys

In [77]:
def weather(x):
    import pprint
    import codecs
    import json
    txt = list()
    session = requests.Session()
    response = session.get(x)
    root = lxml.html.fromstring(response.content)
    import json
    weather = json.loads(root.text)
    return weather

DarkskyからAPIを引っ張ってきます。

In [78]:
def url1():
    import time
    from datetime import datetime
    titi = time.time()
    til = (int(titi/3600))*3600
    T1 = str(til-3600*5)
    return T1

In [79]:
def url2():
    import time
    from datetime import datetime
    titi = time.time()
    til = (int(titi/3600))*3600
    T2 = str(til-3600*7)
    return T2

In [80]:
tokyo ="https://api.darksky.net/forecast/d127d68fbe61cdeb3f662cc97832882b/35.6528,-139.8395,"
osaka="https://api.darksky.net/forecast/d127d68fbe61cdeb3f662cc97832882b/34.6938,-135.5022,"
url1 = url1()
url2 = url2()
w1 = weather(tokyo+url1)
w2 = weather(tokyo+url2)
wo1 = weather(osaka+url1)
wo2 = weather(osaka+url2)

1日分の天候と気温、降雨を取ってくる関数を組みます。

In [81]:
def spt(x):
    ww2 = str(x)
    w3=ww2.split(':')
    w4 = str(w3).split(',')
    return w4

In [82]:
def ap(x,y):
    for i in range(len(x)):
        x1 = x[i]
        x2 = str(x1)[3:]
        y.append(float(x2))
    return y

In [83]:
time = list()
icon = list()
temp = list()
PI = list()
lists = [time, icon, temp, PI]

In [84]:
def wh(x):
    import time
    from datetime import datetime
    import numpy as np
    time = list()
    icon = list()
    temp = list()
    PI = list()
    lists = [time, icon, temp, PI]
    ww = x['hourly']
    wi = str(ww)
    wim = wi.split('{')
    from pandas import DataFrame
    i= 0
    q = 0
    for i in range(len(wim)):
        w4 = spt(wim[i])
        for q in range(len(w4)):
            w = w4[q]
            if 'time' in w:
                time.append(w4[q+1])
            if 'icon' in w:
                icon.append(w4[q+1])
            if 'temp' in w:
                temp.append(w4[q+1])
            if 'precipIntensity' in w:
                PI.append(w4[q+1])
            else:
                pass
    t1 = list()
    for i in range(len(time)):
        t = time[i]
        t2 = str(t)[3:]
        t3 = datetime.fromtimestamp(int(t2))
        t1.append(t3)
    i1 = list()
    i= 0
    for i in range(len(icon)-1):
        a = icon[i+1]
        a2 = str(a)[4:-1]
        i1.append(a2)
    tm = list()
    tm2 = ap(temp,tm)
    pi = list()
    pi2 = ap(PI,pi)
    df = DataFrame({'時間':t1, '天候' : i1, '気温' : tm2, '降雨':pi2})
    return df

In [85]:
ww1 = wh(w1)[7:]

In [86]:
ww2 = wh(w2)[:7]

In [87]:
ww3 = wh(wo1)[7:]

In [88]:
ww4 = wh(wo2)[:7]

In [89]:
import pandas as pd
wh3 = pd.concat([ww1,ww2])

In [90]:
import pandas as pd
wh5 = pd.concat([ww3,ww4])

In [91]:
def whc(x,y):
    import pandas as pd
    from pandas import DataFrame
    a1 = wh(x)[7:]
    a2 = wh(y)[:7]
    a3 = pd.concat([a1,a2])
    return a3

In [92]:
whc = whc(w1,w2)

毎時の指定データを取ってくる関数が組めたので、これをPyMySQLを使ってMySQLに入れたいと思います。

In [93]:
import sqlite3
dbname = 'database.db'
conn = sqlite3.connect(dbname)
c = conn.cursor()

create_table = '''create table wat (time int, icon varchar(64),
                  temperature int, precipIntensity int)'''
c.execute(create_table)

create_table = '''create table wato (time int, icon varchar(64),
                  temperature int, precipIntensity int)'''
c.execute(create_table)

In [94]:
ti = list(whc['時間'])
cl =list(whc['天候'])
te =list(whc['気温'])
ra = list(whc['降雨'])

In [95]:
i = 0
g1 = list()
for i in range(len(ti)):
    g = list()
    g = list([ti[i],cl[i],te[i],ra[i]])
    g1.append(g)

In [96]:
tio = list(wh5['時間'])
clo =list(wh5['天候'])
teo =list(wh5['気温'])
rao = list(wh5['降雨'])

In [97]:
i = 0
g2 = list()
for i in range(len(tio)):
    g = list()
    g = list([tio[i],clo[i],teo[i],rao[i]])
    g2.append(g)

In [98]:
insert_sql = 'insert into wat (time, icon, temperature, precipIntensity) values (?,?,?,?)'
i= 0
for i in range(len(g1)):
    g = g1[i]
    tt =str(ti[i])
    users = [tt, g[1],g[2],g[3]]
    c.execute(insert_sql, users)
    conn.commit()

In [99]:
insert_sql = 'insert into wato (time, icon, temperature, precipIntensity) values (?,?,?,?)'
i= 0
for i in range(len(g2)):
    g = g2[i]
    tt =str(tio[i])
    users = [tt, g[1],g[2],g[3]]
    c.execute(insert_sql, users)
    conn.commit()

In [100]:
select_sql = 'select * from wat'
for row in c.execute(select_sql):
    print(row)

(0, '0', 0, 0)
(0, '0', 0, 0)
(0, 'a', 0, 0)
(0, 'a', 0, 0)
('2017-11-17 00:00:00', 'partly-cloudy-night', 57.34, 0.0003)
('2017-11-16 01:00:00', 'partly-cloudy-night', 60.32, 0.0007)
('2017-11-16 02:00:00', 'partly-cloudy-day', 60.75, 0.0008)
('2017-11-16 03:00:00', 'partly-cloudy-day', 61.6, 0.0008)
('2017-11-16 04:00:00', 'partly-cloudy-day', 61.87, 0.0008)
('2017-11-16 05:00:00', 'partly-cloudy-day', 61.93, 0.0008)
('2017-11-16 06:00:00', 'partly-cloudy-day', 62.08, 0.0007)
('2017-11-16 07:00:00', 'partly-cloudy-day', 61.99, 0.0005)
('2017-11-16 08:00:00', 'partly-cloudy-day', 61.61, 0.0003)
('2017-11-16 09:00:00', 'clear-day', 61.25, 0.0003)
('2017-11-16 10:00:00', 'clear-day', 60.45, 0.0003)
('2017-11-16 11:00:00', 'partly-cloudy-day', 59.67, 0.0005)
('2017-11-16 12:00:00', 'partly-cloudy-night', 59.23, 0.0008)
('2017-11-16 13:00:00', 'partly-cloudy-night', 58.92, 0.0008)
('2017-11-16 14:00:00', 'partly-cloudy-night', 58.67, 0.0008)
('2017-11-16 15:00:00', 'partly-cloudy-night', 

In [101]:
select_sql = 'select * from wato'
for row in c.execute(select_sql):
    print(row)

('2017-11-16 01:00:00', 'partly-cloudy-day', 61.39, 0)
('2017-11-16 02:00:00', 'partly-cloudy-day', 62.09, 0)
('2017-11-16 03:00:00', 'partly-cloudy-day', 62.85, 0.0001)
('2017-11-16 04:00:00', 'partly-cloudy-day', 63.39, 0.0001)
('2017-11-16 05:00:00', 'partly-cloudy-day', 63.78, 0.0001)
('2017-11-16 06:00:00', 'partly-cloudy-day', 64.11, 0.0001)
('2017-11-16 07:00:00', 'partly-cloudy-day', 64.23, 0.0001)
('2017-11-16 08:00:00', 'partly-cloudy-day', 64.05, 0)
('2017-11-16 09:00:00', 'partly-cloudy-day', 63.98, 0)
('2017-11-16 10:00:00', 'partly-cloudy-day', 63.33, 0)
('2017-11-16 11:00:00', 'partly-cloudy-night', 62.64, 0)
('2017-11-16 12:00:00', 'partly-cloudy-night', 62.12, 0.0001)
('2017-11-16 13:00:00', 'partly-cloudy-night', 61.83, 0)
('2017-11-16 14:00:00', 'partly-cloudy-night', 61.63, 0)
('2017-11-16 15:00:00', 'partly-cloudy-night', 61.42, 0.0001)
('2017-11-16 16:00:00', 'partly-cloudy-night', 60.89, 0.0002)
('2017-11-16 17:00:00', 'partly-cloudy-night', 60.13, 0.0004)
('2017

In [102]:
conn.close()

これで、東京と大阪のデータを取ってきて、DBに格納することができました！